# Now we will test an altering-size Q-dict

## Lets first clarify how this is being approached: 

### **1. Arrangement of object balls for a given state is does not matter**

It is important to note that the object balls **do *not* have different values**. Therefore, the arrangment of them on the table does not matter for each unique object ball. They may as well all have the same color and number. For example if we have two object balls, objball_1 at [100,100] and objball_2 at [200,200], it has the same state uniqueness as objball_1 at [200,200] and objball_2 at [100,100].

We can take advantage of this fact as it will reduce the number of unique states by a factor of the number of obj balls on the table. There are not different order permutations for the same set of objball states.

### **2. Obj ball states will be ordered in a sequence to take advantage of not having arrangement uniquess**

We need to have a consistent way to order the states so that a given arrangement is strung the same way. For instance, using the example above, we would be considering the order if we created the following nested array:

> objball_1 = [100,100]
>
> objball_2 = [200,200]
>
> objballs = [objball_1, objball_1] = [ [100,100], [200,200] ]

Since, if the two balls reversed states, we would then have the following nested array:

> objball_1 = [200,200]
>
> objball_2 = [100,100]
>
> objballs = [objball_1, objball_1] = [ [200,200], [100,100] ]

It is clear that these two instances of 'objballs' (the combined state of object balls) are not the same and they should be. To create a consistent ordering wrt obj ball position, it is necessary to point that ordering by distance to x-y origin would cause issues, as well as ordering by slope from origin - two seperate points can have the same slope and two points can also share the same distance to the origin. However, two *seperate* points ***cannot*** have the same slope and distance to the origin. If the have the same slope, they will lie on the same ray pointing to origin and if they have the same dist to origin, they will lie on the same circle with radius of that distance centered about the origin.

Given this, we need to order the 2D states on a 1D line using one or more values that make them unique. Using Cartesean coordinates or cylindrical coordinates requires that the ordering must happen in sequence. We can order by distance from x-axis (using value of y) and then we can further order by distance from y-axis (using value of x)... right? Well, to be honest, *I am not sure* and would want to analyze this further. However, upon deciding how to do this, I came accross a special named ***Row- and column-major order***, which is equivalent to equivalent to ***lexicographic and colexicographic orders***, respectively.

Using this, not only can we map the 2D object ball states with order on a 1D line, we can use this to decrease the memory size of the q-dict. Instead of using a string of 2D cartesean coordinates, we can use their 1D integer mapping. We can do this for the cue ball as well.

### **3. Balls must "snap" to the nearest position discretization.**

In order for discretization to actually help with training, the game must allow for balls to "snap" to nearest discrete positon on simulated billiards table. This means that start and end states for every ball will be within the states specific in the configuration parameters. This must happen in a sequence, so that the snapping to nearest discrete position does not let two balls overlap and create a collision. Furthermore, the sequence in which this happens must preserve state-transition dynamics - otherwise, the stochastic nature of snapping to discrete positions will likely not allow the Q-Learning process to capture long-horizon value. This can be done by snapping the first ball on the table until we get to the last one (the cue ball).

### **4. Reducing memory size on each q-dict**

This is something that has needed quite a lot of thought, since I wanted a way to store state-action Q-values in a way that is compact, fast, and can store a lot. Using Python dictionaries seemed like the way to go, though other data structure/storage options may work even better. 

As for Python dictionaries, it looks like they can run into hash-collisions after a certain size. Therefore, reducing the memory it takes to store this is imperitive. Here are the ways this is being approached:

- #### **4.A Decreasing the string size written to a Q-dict**

    - **2D -> 1D position mappings**:
    
        As mentioned above, we can use a mapping to go from 2D position arrays to 1D scalors. This will help with reducing the amount of integers listed in the string. However, this may not be effective alone, since the value of the 1D mapping may be very large. In order to decrease this number to less characters, we can then map the base-10 integer value to a larger base, such as hex.

    - **Minimizing size of string for each q-dict entry**:

        Initially values in the q-dicts were stored like:

        > {f'{cueball_s}, {objballs_s}, {angle}, {power}': reward}

        Unraveling above would look like:

        > {'[5, 20], [ [50, 80], [90, 10], ..., [25, 100] ], 56.78, 10': 110.00}

        After mapping 2D position values to 1D would look like:
        
        > {'876, [ 25, 98, ..., 1035 ], 56.78, 10': 110.00}

        After mapping 1D position values to hex would look like:
        
        > {'36c, [ 19, 62, ..., 40b ], 56.78, 10': 110.00}

        Now we can add a delimeter to denote cueball state, objball states, theta, and power.Let's use "|" to denote this. For each objball position in the objballs list, we can seperate them with a "," delimiter.

        > {'36c|19,62,...,40b|56.78|10': 110.00}

- #### **4.B Using a q-dict for each different size of object ball sets (0-15)**

    One way to help preserve the size of each q-dict is to have 16 seperate dicts for each size set of object balls on the table, ranging from 0 to 15. Therefore, there will be 16 q-dicts in total and the size of the *obj_s_list* list will determine which q-dict will be used.

In [53]:
hex_val = hex(234587).lstrip("0x")
print(f"hex_val: {hex_val}")
hex_val = f"{hex_val}"

dec_val = int(hex_val, 16)
print(f"dec_val: {dec_val}")

hex_val: 3945b
dec_val: 234587


#### Converting 1D states, actions, and reward to q-string

In [147]:
# States will need to be mapped to 1D to get here
cue_s = 245
obj_s_list = [37, 98, 100, 55, 15, 14, 87, 9, 28, 81, 1, 3, 13, 76, 1035]
obj_s_list.sort() # NEED TO SORT OBJ STATES
angle = 34.98
power = 10000

cue_s_hex = hex(cue_s).lstrip("0x")
obj_s_list_hex = [hex(obj_state).lstrip('0x') for obj_state in obj_s_list]

print("\nHex conversion of Q-table item string:")
SA_pair = f"{cue_s_hex}|"
for obj_s_hex in obj_s_list_hex:
    SA_pair += obj_s_hex + ","
SA_pair = SA_pair.rstrip(",") + f"|{angle}|{power}"

print(SA_pair)


Hex conversion of Q-table item string:
f5|1,3,9,d,e,f,1c,25,37,4c,51,57,62,64,40b|34.98|10000


#### Converting q-string back to decimal

In [128]:
q_values = '36c|19,62,40b|56.78|10'.split("|")
print(f"Splitting q-string:\n {q_values} \n")

cue_s_hex = q_values[0]
obj_s_list_hex = q_values[1]
angle = q_values[2]
power = q_values[3]
print(f"Hexidecimal string:\ncue_s: {cue_s_hex}, obj_states: {obj_s_list_hex}, angle: {angle}, power: {power}\n")

# Convert hex to decimal
cue_s = int(q_values[0], 16)
obj_s_list = [int(obj_state, 16) for obj_state in q_values[1].split(",")]
angle = q_values[2]
power = q_values[3]
print(f"Decimal String:\ncue_s: {cue_s}, obj_states: {obj_s_list}, angle: {angle}, power: {power}\n")


Splitting q-string:
 ['36c', '19,62,40b', '56.78', '10'] 

Hexidecimal string:
cue_s: 36c, obj_states: 19,62,40b, angle: 56.78, power: 10

Decimal String:
cue_s: 876, obj_states: [25, 98, 1035], angle: 56.78, power: 10



#### Test new methods for Q_TABLE class

In [2]:
import numpy as np

In [3]:
# Make a function to map 2D states to 1D
# Will make use of NumPys nd arrays

class MAPPER():
    def __init__(self):
        states_x_list = np.linspace(95, 1140, num = (1140-95)+1)
        states_y_list = np.linspace(0, 618, num = (618-95)+1)
        row_list = []
        num = 1
        for _ in states_y_list:
            row = []
            for _ in states_x_list:
                row.append(num)
                num += 1
            row_list.append(row)
        self.x = np.array(row_list, np.int32, order='F')

    def map_pos_to_1D(self, pos):
        min_x = 95
        max_x = 1140
        min_y = 95
        max_y = 618
        if pos[0] < min_x or pos[0] > max_x or pos[1] < min_y or pos[1] > max_y:
            raise Exception(f"The position [{pos[0]}, {pos[1]}] is out of bounds!"
                            f" Please note the min pos is [{min_x}, {min_y}] and the max pos is [{max_x}, {max_y}].")
        return self.x[pos[1]-min_x, pos[0]-min_y]

In [4]:
# Testing mapping class

mapper = MAPPER()
pos = [95,95]
mapped_pos = mapper.map_pos_to_1D(pos)
print(mapped_pos)

1


In [5]:
def convert_to_hexstring(cue_s, obj_s_list, angle, power):
    if len(obj_s_list) > 0:
        obj_s_list.sort()
        cue_s_hex =      hex(cue_s).lstrip("0x")
        obj_s_list_hex = [hex(obj_state).lstrip('0x') for obj_state in obj_s_list]
        SA_pair =        f"{cue_s_hex}|"
        for obj_s_hex in obj_s_list_hex:
            SA_pair += obj_s_hex + ","
        SA_pair = SA_pair.rstrip(",") + f"|{angle}|{power}"
        return SA_pair

In [10]:
class Q_DICT:
    def __init__(self):
        mapper = MAPPER()
        cue_s_start = mapper.map_pos_to_1D([95, 95])
        obj_s_start = []
        theta_start = 0.00
        power_start = 0
        reward = 0.00

        self.Q_dict_pointer = None
        self.Q_dict_list = [{} for _ in range(16)]
        for i, q_dict in enumerate(self.Q_dict_list):
            setattr(self, f'Q_dict_{i}', q_dict)
        
        for _ in range(16):
            SA_string =convert_to_hexstring(cue_s_start, obj_s_start, theta_start, power_start)
            self.Q_dict_pointer = {SA_string: reward}
            self.Q_dict_list[len(obj_s_start)] = self.Q_dict_pointer
            obj_s_start.append(mapper.map_pos_to_1D([95, 95]))
            #print(self.Q_dict_list)

    # TODO: Should I still use this?
    def get_q_dict(self, obj_s):
        return self.Q_dict_list[len(obj_s)]
        #return self.Q_dict_pointer
    
#max_power = 20000
#powers = np.linspace(0, max_power, num = 41) # List from 0 to max_power, with powers_disc numbers
#print(powers)

In [11]:
Q = Q_DICT()

for Q_dict in Q.Q_dict_list:
    #print(Q_dict_list)
    if None not in Q_dict.keys():
        q_values = [key_val for key_val in Q_dict.keys()][0].split("|")
        #print(f"Hexidecimal string:\n{q_values}")
        #print("")

        # Convert hex to decimal
        cue_s = int(q_values[0], 16)
        obj_s_list = [int(obj_state, 16) for obj_state in q_values[1].split(",")]
        angle = q_values[2]
        power = q_values[3]
        print(f"Decimal String:\ncue_s: {cue_s}, obj_states: {obj_s_list}, angle: {angle}, power: {power}\n")


Decimal String:
cue_s: 1, obj_states: [1], angle: 0.0, power: 0

Decimal String:
cue_s: 1, obj_states: [1, 1], angle: 0.0, power: 0

Decimal String:
cue_s: 1, obj_states: [1, 1, 1], angle: 0.0, power: 0

Decimal String:
cue_s: 1, obj_states: [1, 1, 1, 1], angle: 0.0, power: 0

Decimal String:
cue_s: 1, obj_states: [1, 1, 1, 1, 1], angle: 0.0, power: 0

Decimal String:
cue_s: 1, obj_states: [1, 1, 1, 1, 1, 1], angle: 0.0, power: 0

Decimal String:
cue_s: 1, obj_states: [1, 1, 1, 1, 1, 1, 1], angle: 0.0, power: 0

Decimal String:
cue_s: 1, obj_states: [1, 1, 1, 1, 1, 1, 1, 1], angle: 0.0, power: 0

Decimal String:
cue_s: 1, obj_states: [1, 1, 1, 1, 1, 1, 1, 1, 1], angle: 0.0, power: 0

Decimal String:
cue_s: 1, obj_states: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], angle: 0.0, power: 0

Decimal String:
cue_s: 1, obj_states: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], angle: 0.0, power: 0

Decimal String:
cue_s: 1, obj_states: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], angle: 0.0, power: 0

Decimal String:
cue_s:

In [1]:
import numpy as np
from tqdm import tqdm

In [677]:
class Q_DICT:
    def __init__(self):
        cueball_s_start=[0, 0]
        objball_s_start=[0, 0]
        angle_start=0.00
        power_start=0
        reward=0.00
        self.Q_dict = {f'{cueball_s_start}, {objball_s_start}, {angle_start}, {power_start}': reward}

    def update_q_dict(self, cue_state, target_state, angle, power, reward):
        current_reward = self.get_q_val(cue_state, target_state, angle, power)
        if current_reward is not None:
            updated_reward = current_reward + reward
        else:
            updated_reward = reward       
        self.Q_dict.update({f'{cue_state}, {target_state}, {angle}, {power}': updated_reward})     

    def get_q_val(self, cue_state, target_state, angle, power):
        state_action_pair = f'{cue_state}, {target_state}, {angle}, {power}'
        if state_action_pair in self.Q_dict: 
            current_val = self.Q_dict[state_action_pair]
        else:
            current_val = None
        return current_val

# Initialize the Q-table as a Python dict

## a) Init full Action and State Space (Will take FOREVER)

In [58]:
'''
Q = Q_DICT()

Q.Q_dict = init_q_dict(x_disc=501, y_disc=501, angles_disc=361, powers_disc=5)
'''

'\nQ = Q_DICT()\n\nQ.Q_dict = init_q_dict(x_disc=501, y_disc=501, angles_disc=361, powers_disc=5)\n'

## b) Initialize Q-table and update as we see states (faster, but may not have an action for an unseen state-action pair)

In [66]:
Q = Q_DICT()

print(Q.Q_dict)

{'[0.0, 0.0], [0.0, 0.0], 0.0, 0.0': 0.0}


In [67]:
cue_state = [1.0, 0.0]
target_state = [0.0, 1.0]
angle = 0.00
power = 0.00
reward = 10.00

Q.update_q_dict(cue_state, target_state, angle, power, reward)

print(Q.Q_dict)

{'[0.0, 0.0], [0.0, 0.0], 0.0, 0.0': 0.0, '[1.0, 0.0], [0.0, 1.0], 0.0, 0.0': 10.0}


# Check Q-table (Q-dict) to see what the latest key is

In [68]:
print(list(Q.Q_dict)[-1])

[1.0, 0.0], [0.0, 1.0], 0.0, 0.0


# Check to see what the Q-value is for a given key

In [69]:
# Check what reward is for a specific state-action pair:
cue_state = [1.00, 0.00]
target_state = [0.00, 1.00]
angle = 0.00
power = 0.00

q_val = Q.get_q_val(cue_state, target_state, angle, power)
print(q_val)

10.0


In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
